In [25]:
import numpy as np
import pandas as pd
from numpy import array
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split,cross_val_score,cross_validate,GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, multilabel_confusion_matrix, classification_report

In [26]:
df = pd.read_csv("SpotifyWithDate.csv")

# Normalize data
X, y = df.drop(columns=['popularity', 'track_id']), df['popularity']
X = (X-X.min())/(X.max()-X.min())
y = ((y-1)//20)+1

print(X)
print(y)

        danceability  instrumentalness  liveness  loudness  speechiness  \
0           0.452158          0.783784  0.147759  0.744092     0.015030   
1           0.666344          0.000688  0.036988  0.814074     0.008467   
2           0.697557          0.000856  0.091212  0.807442     0.017252   
3           0.670649          0.000000  0.216423  0.828429     0.026778   
4           0.392961          0.000005  0.099290  0.803454     0.011960   
...              ...               ...       ...       ...          ...   
122658      0.904208          0.002052  0.069401  0.740785     0.056202   
122659      0.132494          0.930931  0.095251  0.388209     0.015982   
122660      0.179851          0.112112  0.058900  0.377512     0.022227   
122661      0.677107          0.000000  0.197237  0.567600     0.940728   
122662      0.527500          0.000052  0.229550  0.777741     0.154318   

         valence      year  
0       0.110457  0.966942  
1       0.849883  0.909091  
2       0.61

In [27]:
def getdistribution(data):
    distribution = dict()
    for value in data:
        if value in distribution:
            distribution[value] += 1
        else:
            distribution[value] = 1
    return distribution

dist = getdistribution(y)
print(dist) # print frequenc


{5: 534, 4: 9622, 3: 40131, 2: 59120, 1: 13256}


In [33]:
# It will take hours to run
gridSearch = GridSearchCV(MLPClassifier(solver = 'sgd', random_state=1), {
    'activation':["relu"],
    'hidden_layer_sizes': [(19,35),(20,17)],
    'learning_rate_init':[0.02,0.03],
    'max_iter': [400]
}, cv=10, return_train_score=False)
test_result = gridSearch.fit(X, y)
print(test_result)
print(test_result.best_score_)
print(test_result.best_params_)

GridSearchCV(cv=10, estimator=MLPClassifier(random_state=1, solver='sgd'),
             param_grid={'activation': ['relu'],
                         'hidden_layer_sizes': [(19, 35)],
                         'learning_rate_init': [0.02], 'max_iter': [400]})
0.5149969386647395
{'activation': 'relu', 'hidden_layer_sizes': (19, 35), 'learning_rate_init': 0.02, 'max_iter': 400}


In [32]:
tuning_df = pd.DataFrame(gridSearch.cv_results_)
tuning_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate_init,param_max_iter,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,19.377186,5.479691,0.005359,0.001444,relu,"(12, 8)",0.05,400,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.470775,...,0.518833,0.503669,0.522827,0.510680,0.529920,0.505625,0.454508,0.502352,0.022242,3
1,24.571468,11.499931,0.005180,0.001066,relu,"(12, 8)",0.02,400,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.478438,...,0.498369,0.510435,0.516631,0.505544,0.522420,0.484021,0.453122,0.497102,0.019391,4
2,21.005482,3.130535,0.009286,0.001175,relu,"(19, 35)",0.05,400,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.485693,...,0.523724,0.519811,0.523072,0.510109,0.519892,0.495271,0.455976,0.504512,0.020066,2
3,55.430889,13.575087,0.010303,0.001420,relu,"(19, 35)",0.02,400,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.449906,...,0.531551,0.523561,0.530735,0.526251,0.541334,0.533670,0.487363,0.514997,0.026048,1
